In [202]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from collections import deque
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

64-bit Processor,<br>
32 registers and each register is 64-bit<br>

Functional Units<br>
Integer Addition/Subtraction (IADD Rdst, Rsrc1, Rsrc2)  -  6 CC<br>
Integer Multiplication (IMUL Rdst, Rsrc1, Rsrc2)  -  12 CC<br>
Floating point Addition/Subtraction (FADD Rdst, Rsrc1, Rsrc2)  -  18 CC<br>
Floating point Multiplication (FADD Rdst, Rsrc1, Rsrc2)  -  30 CC<br>
Load (LD Rdst, Mem) - 1 CC<br>
Store (ST Mem, Rsrc) - 1 CC<br>
Logic Unit (AND/OR/XOR Rdst, Rsrc1, Rsrc2)  -  1 CC<br>
No Operation (NOP) 1CC<br>

In [227]:
class Tomasulo:
    def __init__(self):
        self.clock = 0
        r_list = [f"r{i}" for i in range(32)]
        self.register_file = pd.DataFrame({r_list[i]:[[],-1] for i in range(32)}) 
        self.fus = ['iadd', 'imul', 'fadd', 'fmul', 'load', 'store', 'logic']
        self.clock_cycles = [6, 12, 18, 30, 1, 1, 1]
        self.instruction_queue = deque()
        self.common_data_bus = []
        self.reservation_stations = {self.fus[i]:[0,0,0,0,0,0] for i in range(len(self.fus))}
        self.num_reservation_stations = {self.fus[i]:3 for i in range(len(self.fus))}
        self.cc = {self.fus[i]:self.clock_cycles[i] for i in range(len(self.clock_cycles))}
        self.memory = {x:5 for x in range(65536)}
        self.instruction_table = pd.DataFrame(columns = ['cc', 'Instr','Issue', 'Execute', 'Write Result'])

    def check_status(self):
        print("Reservation Stations:")
        for key, value in self.reservation_stations.items():
            print(f"{key}: {value}")

        print("\nInstruction Table:")
        print(self.instruction_table)
        
    def execute(self, instruction, decode_string):
        result = 0
        if(decode_string == 'fp'):
            self.clock += self.cc[instruction[0]]
            self.reservation_stations[instruction[0]][0] = 0
            self.reservation_stations[instruction[0]][1] = 0

            result = self.compute_result(instruction[0], self.register_file[instruction[1]][1], self.register_file[instruction[2]][1])

        elif(decode_string == 'load'):
            self.clock += self.cc[instruction[0]]
            self.reservation_stations[instruction[0]][0] = 0
            self.reservation_stations[instruction[0]][4] += self.reservation_stations[instruction[0]][2]
            self.reservation_stations[instruction[0]][4] = self.memory[self.reservation_stations[instruction[0]][4]]

        else:
            self.clock += self.cc[instruction[0]]
            self.reservation_stations[instruction[0]][0] = 0
            self.reservation_stations[instruction[0]][4] += self.reservation_stations[instruction[0]][2]

        row_index = len(self.instruction_table)
        self.instruction_table.loc[row_index, ["cc", "Instr", "Issue", "Execute", "Write Result"]] = [
            self.clock, instruction[0], 'done', 'done', ''
        ]
        self.check_status()
        self.write_result(instruction, result, decode_string)

    def instruction_fetch(self, instructions):
        for row in instructions.itertuples(index=False):
            self.instruction_queue.append(list(row))

        while self.instruction_queue:
            issued_instructions = []
            
            # Try to issue multiple instructions in one cycle
            for _ in range(len(self.instruction_queue)):
                temp_lst = self.instruction_queue.popleft()
                if self.issue(temp_lst):
                    issued_instructions.append(temp_lst)
                    row_index = len(self.instruction_table)
                    self.instruction_table.loc[row_index, ["cc", "Instr", "Issue", "Execute", "Write Result"]] = [
                        self.clock, temp_lst[0], 'done', '', ''
                    ]
            self.clock += 1  # Advance the clock after issuing in this cycle

            # Execute only after issuing stage is complete
            for instr in issued_instructions:
                self.execute(instr, self.decode(instr))

    
    def decode(self, instruction):
        if(instruction[0] == 'load'):
            return 'load'
        if(instruction[0] == 'store'):
            return 'store'
        
        return 'fp'
    
    def issue(self, instruction):
        decode_string = self.decode(instruction)
        
        if self.num_reservation_stations[instruction[0]] <= 0:
            self.instruction_queue.append(instruction)  
            return False 

        # self.clock += 1 
        
        # Issue logic remains the same
        if decode_string == 'fp':
            if self.register_file[instruction[2]][0] != []:
                self.reservation_stations[instruction[0]][0] = self.register_file[instruction[2]][0][1]
            else:
                self.reservation_stations[instruction[0]][2] = int(self.register_file[instruction[2]][1])
                self.reservation_stations[instruction[0]][0] = 0

            if self.register_file[instruction[3]][0] != []:
                self.reservation_stations[instruction[0]][1] = self.register_file[instruction[3]][0][1]
            else:
                self.reservation_stations[instruction[0]][3] = int(self.register_file[instruction[3]][1])
                self.reservation_stations[instruction[0]][1] = 0

            self.reservation_stations[instruction[0]][5] = 'busy'
            self.register_file[instruction[1]][0] = [instruction[0], self.num_reservation_stations[instruction[0]]]
            self.num_reservation_stations[instruction[0]] -= 1

        else:
            if self.register_file[instruction[3]][0] != []:
                self.reservation_stations[instruction[0]][0] = self.register_file[instruction[3]][0][1]
            else:
                self.reservation_stations[instruction[0]][2] = self.register_file[instruction[3]][1]
            
            self.reservation_stations[instruction[0]][4] = int(instruction[2])
            self.reservation_stations[instruction[0]][5] = 'busy'
            self.num_reservation_stations[instruction[0]] -= 1

            if decode_string == 'load':
                self.register_file[instruction[1]][0] = [instruction[0], self.num_reservation_stations[instruction[0]]]
            else:
                if self.register_file[instruction[1]][0] != []:
                    self.reservation_stations[instruction[0]][1] = self.register_file[instruction[1]][0][1]
                else:
                    self.reservation_stations[instruction[0]][3] = int(self.register_file[instruction[1]][1])
                    self.reservation_stations[instruction[0]][1] = 0

        return True 


    def compute_result(self, operation, value1, value2):
        value1 = int(value1)
        value2 = int(value2)
        if(operation == 'iadd'):
            return value1 + value2
        if(operation == 'imul'):
            return value1 * value2
        if(operation == 'fadd'):
            return value1 + value2
        if(operation == 'fmul'):
            return value1 * value2
        
    def write_result(self, instruction, result, decode_string):

        if(decode_string == 'fp' or decode_string == 'load'):
            for key, value in self.register_file.items():
                # print(f"Checking {key}: {type(value)}")
                value = list(value)
                if(value[0] != []):
                    if(value[0][0] == decode_string):
                        self.register_file[key][1] = result
                        self.register_file[key][0] = []
            
            self.reservation_stations[instruction[0]][2] = int(result)
            self.reservation_stations[instruction[0]][0] = 0

            self.reservation_stations[instruction[0]][3] = int(result)
            self.reservation_stations[instruction[0]][1] = 0
        else:
            self.memory[self.reservation_stations[instruction[0]][4]] = self.reservation_stations[instruction[0]][3]
        
        self.reservation_stations[instruction[0]][5] = ''
        self.num_reservation_stations[instruction[0]] += 1
        row_index = len(self.instruction_table)
        self.instruction_table.loc[row_index, ["cc", "Instr", "Issue", "Execute", "Write Result"]] = [
            self.clock, instruction[0], 'done', 'done', 'done'
        ]
        self.check_status()

In [228]:
def test_program(program):
    print("Code segment:")
    for row in program.itertuples(index = True):
        print("Line:",row[0],"\t",row[1:])

In [229]:
def run(program):
    toma = Tomasulo()
    toma.instruction_fetch(program)
    return 0

In [230]:
if __name__ == '__main__':
    '''TOMASULO IMPLEMENTATION'''
    '''PROCESSOR DETAILS
        16 bit processor (32 16-bit Registers)

        Number of clock cycles :
            Integer Addtion: 6 cc
            Integer Multiplication: 12 cc
            Floating Point Addition: 18 cc
            Floating Point Multiplication: 30 cc
            Load unit: 1 cc
            Store unit: 1 cc
            Logic unit: 1cc

        Has multiple reservation stations per Functional unit that enables register renaming and thus prevents WAR and WAW hazard.
    '''

    #Read the Program
    program = pd.read_csv('program.txt', header = None)
    #print the Program
    test_program(program)

    #Run the Program using Tomasulo Processor
    exit_status = run(program)

    print(f"The program has been executed successfully with code {exit_status}.")

Code segment:
Line: 0 	 ('load', 'r0', '32', 'r2')
Line: 1 	 ('load', 'r4', '32', 'r2')
Line: 2 	 ('load', 'r2', '44', 'r3')
Line: 3 	 ('imul', 'r0', 'r2', 'r4')
Line: 4 	 ('iadd', 'r8', 'r2', 'r6')
Line: 5 	 ('fmul', 'r10', 'r0', 'r6')
Line: 6 	 ('fadd', 'r6', 'r8', 'r2')
Reservation Stations:
iadd: [0, 0, 0, -1, 0, 'busy']
imul: [0, 1, 0, 0, 0, 'busy']
fadd: [3, 0, 0, 0, 0, 'busy']
fmul: [3, 0, 0, -1, 0, 'busy']
load: [0, 0, -1, 0, 5, 'busy']
store: [0, 0, 0, 0, 0, 0]
logic: [0, 0, 0, 0, 0, 0]

Instruction Table:
  cc Instr Issue Execute Write Result
0  0  load  done                     
1  0  load  done                     
2  0  load  done                     
3  0  imul  done                     
4  0  iadd  done                     
5  0  fmul  done                     
6  0  fadd  done                     
7  2  load  done    done             
Reservation Stations:
iadd: [0, 0, 0, -1, 0, 'busy']
imul: [0, 1, 0, 0, 0, 'busy']
fadd: [3, 0, 0, 0, 0, 'busy']
fmul: [3, 0, 0, -1, 0, '